In [30]:
import tensorflow as tf
import pickle
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

In [31]:
### load ANN trained model and scaler pickel
model=load_model('model.h5')

#load scaler and encoder
with open('Data/label_encoder_gender.pkl','rb')as file:
    label_encoder_gender=pickle.load(file)

with open('Data/encode_geo.pkl','rb') as file:
    encode_geo=pickle.load(file)

with open('Data/scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [32]:
# Example customer record with all columns
customer_A = {
    "CreditScore": 619,
    "Geography": "France",
    "Gender": "Female",
    "Age": 42,
    "Tenure": 2,
    "Balance": 0.00,
    "NumOfProducts": 1,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 101348.88
}

In [33]:
## one hot encoded geography
geo=encode_geo.transform([[customer_A['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo,columns=encode_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [34]:

print(encode_geo.get_feature_names_out(['Geography']))

['Geography_France' 'Geography_Germany' 'Geography_Spain']


In [35]:
customer_df=pd.DataFrame([customer_A])
customer_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.0,1,1,1,101348.88


In [36]:
## encode categorical var
customer_df['Gender']=label_encoder_gender.transform(customer_df['Gender'])
customer_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.0,1,1,1,101348.88


In [37]:
# combine one hot encoding with input data
customer_df=pd.concat([customer_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
customer_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.0,1,1,1,101348.88,1.0,0.0,0.0


In [38]:
### Scaling the input data
input_scaled=scaler.transform(customer_df)
input_scaled

array([[-0.472045  , -1.13761448,  0.43732866, -1.07596467, -0.88465622,
        -1.01401427,  0.57144077,  1.00541507, -0.22545415,  0.86727152,
        -0.51576047, -0.52978452]])

In [39]:
## Predict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


array([[0.47537923]], dtype=float32)

In [40]:
prediction_prob=prediction[0][0]
prediction_prob

0.47537923

In [41]:
if(prediction_prob>0.5):
    print('The customer is likely to churn')
else:
    print('The customer is  not likely to churn')

The customer is  not likely to churn
